# Gradiente Ascendente con Vectores

En este notebook vamos a implementar **el algoritmo de Gradiente Ascendente** aplicado a la **regresión logística**.  
La idea es aprender a *maximizar* la **verosimilitud** del modelo ajustando los parámetros (pesos) del vector β.

Aprenderemos:
- Qué son los vectores y matrices en NumPy.
- Cómo se calcula el **producto punto**.
- Qué hace la **función sigmoide**.
- Cómo funciona el **gradiente ascendente** paso a paso.

---


## Operaciones básicas con vectores y matrices

In [ ]:
# importamos la libreria numpy
import numpy as np # Para hacer calculos numéricos


# 1. VECTORES

# definimos vectores (arreglos 1D)
v1 = np.array([1,2,3])
v2 = np.array([1,0,1])
print("Vector 1:", v1)
print("Vector 2:", v2)

# Producto punto (dot product)
# Multiplica elemento a elemento y luego suma los resultados:
#  (1*1) + (2*0) + (3*1) = 4
a = np.dot(v1,v2)
print("Producto punto:", a)

# Producto cruz (cross product)
# Devuelve un nuevo vector perpendicular a los dos vectores de entrada
b = np.cross(v1,v2)
print("Producto cruz:", b)


# 2. MATRICES


# Creamos una matriz 3x4 (3 renglones, 4 columnas)
m1 = np.array([
      [1,2,3,0],
      [4,5,6,1],
      [7,8,9,8]
])

# Creamos una matriz 4x3
m2 = np.array([
      [1,2,3],
      [4,5,6],
      [7,8,9],
      [10,11,12]
])

print("Matriz 1:")
print(m1)
print("\nMatriz 2:")
print(m2)

# Multiplicamos matrices
# Regla: (n×m) * (m×k) = (n×k)
# Aquí: (3x4) * (4x3) = (3x3)
m3 = m1 @ m2 # @ es un operador de numpy para mult de matrices
print("\nMatriz multiplicada m3:")
print(m3)

# La transpuesta invierte renglones por columnas
m3t = m3.T
print("\nTranspuesta de m3:")
print(m3t)


# 5. MATRICES DE CEROS Y UNOS

# Matriz de ceros: llena con 0.0
m4 = np.zeros((3, 5))  # 3 renglones, 5 columnas
print("\nMatriz m4 (ceros):")
print(m4)

# creamos matriz de unos
# Matriz de unos: llena con 1.0
m5 = np.ones((3, 2))   # 3 renglones, 2 columnas
print("\nMatriz m5 (unos):")
print(m5)


# 6. OBTENER DIMENSIONES DE UNA MATRIZ

# La propiedad .shape devuelve una tupla (renglones, columnas)
print("\nForma de m5:", m5.shape) # (3, 2)

# Guardamos los valores de renglones y columnas
r = m5.shape[0]  # número de renglones
c = m5.shape[1]  # número de columnas
print("Renglones:", r)
print("Columnas:", c)

# También se puede "desempaquetar" directamente
r, c = m5.shape
print("\nDesempaquetamiento de tupla .shape:")
print("Renglones:", r)
print("Columnas:", c)


# 7. REFORMA (RESHAPE) DE MATRICES Y VECTORES

# Podemos cambiar la forma de un vector

# Creamos un vector columna de 4x1 con unos
v4 = np.ones((4,1)) # vector columna (4 renglones, 1 columna)
print("\nVector columna (4x1):")
print(v4)

# Cambiamos su forma a vector fila (1x4)
v5 = v4.reshape((1, 4))
print("\nDespués del reshape (1x4):")
print(v5)

# 8. APILAMIENTO DE MATRICES (STACK)

# Creamos una matriz base 2x2
m6 = np.array([
    [1, 2],
    [3, 4]
])

# Agregamos una columna de unos al inicio
# np.hstack concatena matrices horizontalmente
print("Matriz original m6:")
print(m6)
m7 = np.hstack((np.ones((2, 1)), m6))
print("\nMatriz resultante de m6 después del hstack (columna de unos agregada):")
print(m7)




Vector 1: [1 2 3]
Vector 2: [1 0 1]
Producto punto: 4
Producto cruz: [ 2  2 -2]
Matriz 1:
[[1 2 3 0]
 [4 5 6 1]
 [7 8 9 8]]

Matriz 2:
[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]

Matriz multiplicada m3:
[[ 30  36  42]
 [ 76  92 108]
 [182 214 246]]

Transpuesta de m3:
[[ 30  76 182]
 [ 36  92 214]
 [ 42 108 246]]

Matriz m4 (ceros):
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]

Matriz m5 (unos):
[[1. 1.]
 [1. 1.]
 [1. 1.]]

Forma de m5: (3, 2)
Renglones: 3
Columnas: 2

Desempaquetamiento de tupla .shape:
Renglones: 3
Columnas: 2

Vector columna (4x1):
[[1.]
 [1.]
 [1.]
 [1.]]

Después del reshape (1x4):
[[1. 1. 1. 1.]]
Matriz original m6:
[[1 2]
 [3 4]]

Matriz resultante de m6 después del hstack (columna de unos agregada):
[[1. 1. 2.]
 [1. 3. 4.]]


# Función Sigmoide y Algoritmo de Gradiente Ascendente

In [ ]:

# 1. FUNCIÓN SIGMOIDE

def sigmoide(z):
    """
    Función de activación sigmoide:
    σ(z) = 1 / (1 + e^(-z))

    Convierte cualquier número real (positivo o negativo)
    en un valor dentro del rango (0, 1).

    En regresión logística, esta salida se interpreta como una probabilidad:
    - Valores cercanos a 1 → alta probabilidad de que Y sea 1 (clase positiva).
    - Valores cercanos a 0 → alta probabilidad de que Y sea 0 (clase negativa).
    - Valores alrededor de 0.5 → el modelo está indeciso.

    """

    return 1 / (1 + np.exp(-z))



# 2. FUNCIÓN DE GRADIENTE ASCENDENTE

def grad_asc(Xc, Y, eta=0.1, max_it=10000):
    """
    Entrena un modelo de regresión logística usando **gradiente ascendente**.

    Idea general:
    ----------------
    Ajusta los parámetros β para que las predicciones del modelo
    se acerquen lo más posible a los valores reales observados en Y.

    En otras palabras:
    El algoritmo busca los β que maximizan la probabilidad (verosimilitud)
    de que el modelo haya generado los datos que tenemos.

    Retorna:
    -----------
    beta : np.ndarray (n+1, 1)
        Vector de pesos aprendidos (incluye el sesgo β₀).
    """

    # m = número de ejemplos (filas), n = número de características (columnas)
    m, n = Xc.shape # obtenemos dimensiones de Xc

    # Inicializamos los pesos en cero (β₀ + β₁ ... βn)
    beta = np.zeros((n + 1, 1)) 

    # Agregamos una columna de unos al inicio → representa el sesgo β₀
    X = np.hstack((np.ones((m, 1)), Xc))


    #  Bucle de entrenamiento

    for i in range(max_it):

        # 1. Predicción del modelo: σ(X·β)
        # La sigmoide transforma la combinación lineal X·β en probabilidades (0 a 1)
        p = sigmoide(X @ beta) 
        #X: matriz de características con columna de unos
        #beta: vector de pesos

        # 2️. Cálculo del gradiente:
        # (Y - p) mide el error entre lo real y lo predicho.
        # X.T @ (Y - p) indica hacia dónde mover los β para mejorar las predicciones.
        grad = X.T @ (Y - p)
        #X.T: transpuesta de la matriz de características
        #Y: etiquetas reales
        #p: probabilidades predichas por el modelo
        

        # 3️. Evaluar el tamaño del cambio
        # Si el gradiente es casi 0, significa que ya llegamos al “punto óptimo”.
        norm_grad = np.linalg.norm(grad)

        if norm_grad < 1e-4:
            print(f"✅ Convergencia alcanzada en la iteración {i}")
            break

        # 4️. Actualizamos los parámetros (subimos por la "montaña" de verosimilitud)
        #   beta ← beta + η * grad
        # eta controla el tamaño del paso (tasa de aprendizaje)
        beta = beta + eta * grad

    # Retorna los β finales: los valores que mejor ajustan el modelo
    return beta





# Ejemplo práctico: entrenamiento del modelo

En este ejemplo creamos un conjunto de datos muy simple para probar el algoritmo:

- **Xc** representa las características de entrada (cada fila es un ejemplo y cada columna una variable).  
- **Y** son las *etiquetas reales*, es decir, los valores correctos que el modelo debe aprender a predecir (0 o 1).  
- Luego entrenamos el modelo con `grad_asc()`, que ajusta los pesos **β** para que las predicciones se acerquen lo más posible a los valores reales.

Al final se imprimen los parámetros aprendidos:
- **β₀** es el sesgo o intercepto.  
- **β₁, β₂, ...** son los pesos de cada característica, indicando su influencia en la probabilidad de que la salida sea 1.


In [6]:

# Cada fila representa un ejemplo y cada columna una característica (feature)
Xc = np.array([
    [0, 0, 1, 0],
    [1, 1, 1, 1],
    [0, 0, 0, 1]
])

# Etiquetas reales (valores correctos que el modelo debe aprender a predecir)
# 1 = clase positiva, 0 = clase negativa
Y = np.array([1, 0, 1]).reshape((-1, 1))

# Entrenamiento del modelo: ajusta los pesos β para que las predicciones
# se acerquen lo más posible a las etiquetas reales (Y)
betas = grad_asc(Xc, Y)

# β₀ es el sesgo (intercepto) y los demás β son los pesos de cada característica
print("\nParámetros β aprendidos:")
print(betas)



Parámetros β aprendidos:
[[ 6.90214847e+00]
 [-6.90214847e+00]
 [-6.90214847e+00]
 [-3.91173639e-15]
 [-3.91173639e-15]]
